<a href="https://colab.research.google.com/github/Harsha-Sri24/Cloud/blob/main/Week1_Handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 5588 — Week 1: Hands-On Lab
## Mini-RAG Pipeline: Embeddings → Retrieval → Grounded Generation

**Goals:**
- Generate semantic embeddings using a Transformer-based encoder
- Build a vector index for fast similarity search
- Retrieve top-k relevant document chunks
- Inject retrieved context into an LLM prompt for grounded generation

**Workflow:** GitHub → Colab → Hugging Face → Vector Store (FAISS / Chroma) → LLM

---


### GenAI Systems Context (Mini-RAG)
This lab implements a **mini Retrieval-Augmented Generation (RAG)** pipeline:
- A **Transformer encoder** produces semantic embeddings
- A **vector index (FAISS)** enables fast retrieval
- Retrieved context is what a downstream **LLM** would use for grounded generation


## Step 1 — Environment Setup
Install required libraries. This may take ~1 minute.


In [1]:
!pip install -q transformers datasets sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 29.4 MB/s eta 0:00:00


## Step 2 — Load Dataset & Model from Hugging Face Hub
We use a lightweight news dataset and a sentence embedding model.


In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

dataset = load_dataset("ag_news", split="train[:200]")
model = SentenceTransformer("all-MiniLM-L6-v2")

texts = list(dataset["text"])
print(f"Loaded {len(texts)} documents")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded 200 documents


## Step 3 — Create Embeddings
These vectors represent semantic meaning and enable retrieval before generation.


In [3]:
embeddings = model.encode(texts, show_progress_bar=True)
print('Embedding shape:', embeddings.shape)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Embedding shape: (200, 384)


## Step 4 — Build a Vector Index (FAISS)
This simulates the retrieval layer in RAG systems.


In [4]:
import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
print('Index size:', index.ntotal)

Index size: 200


## Step 5 — Retrieval Function
Search for documents related to a query.


In [5]:
def search(query, k=3):
    q_emb = model.encode([query])
    distances, indices = index.search(np.array(q_emb), k)
    return [texts[i] for i in indices[0]]

## Step 6 — Try It!


In [6]:
search("artificial intelligence in healthcare")

["U.K.'s NHS taps Gartner to help plan \\$9B IT overhaul LONDON -- The U.K.'s National Health Service (NHS) has tapped IT researcher Gartner Inc. to provide market intelligence services as the health organization forges ahead with a mammoth, 5 billion (\\$9.2 billion) project to upgrade its information technology infrastructure.",
 'UK Scientists Allowed to Clone Human Embryos (Reuters) Reuters - British scientists said on Wednesday\\they had received permission to clone human embryos for medical\\research, in what they believe to be the first such license to\\be granted in Europe.',
 'Coming to The Rescue Got a unique problem? Not to worry: you can find a financial planner for every specialized need']

In [7]:
search("stock market and economy")

["Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the economy and the outlook for earnings are expected to  hang over the stock market next week during the depth of the  summer doldrums.",
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Stocks End Up, But Near Year Lows (Reuters) Reuters - Stocks ended slightly higher on Friday\\but stayed near lows for the year as oil prices surged past  #36;46\\a barrel, offsetting a positive outlook from computer maker\\Dell Inc. (DELL.O)']

In [8]:
print(search("artificial intelligence in healthcare"))

["U.K.'s NHS taps Gartner to help plan \\$9B IT overhaul LONDON -- The U.K.'s National Health Service (NHS) has tapped IT researcher Gartner Inc. to provide market intelligence services as the health organization forges ahead with a mammoth, 5 billion (\\$9.2 billion) project to upgrade its information technology infrastructure.", 'UK Scientists Allowed to Clone Human Embryos (Reuters) Reuters - British scientists said on Wednesday\\they had received permission to clone human embryos for medical\\research, in what they believe to be the first such license to\\be granted in Europe.', 'Coming to The Rescue Got a unique problem? Not to worry: you can find a financial planner for every specialized need']


In [9]:
print(search("stock market and economy"))

["Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the economy and the outlook for earnings are expected to  hang over the stock market next week during the depth of the  summer doldrums.", "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Stocks End Up, But Near Year Lows (Reuters) Reuters - Stocks ended slightly higher on Friday\\but stayed near lows for the year as oil prices surged past  #36;46\\a barrel, offsetting a positive outlook from computer maker\\Dell Inc. (DELL.O)']


## Reflection
In 1–2 sentences, explain how embeddings enable retrieval before generation in GenAI systems.
